In [15]:
from faster_whisper import WhisperModel
import os
import re


In [16]:
model_size = "large-v2"
model = WhisperModel(model_size, device="cpu", compute_type="float32")

In [17]:
def extract_number(filename):
    match = re.search(r'audio-(\d+)\.wav', filename)
    if match:
        return int(match.group(1))
    return -1


In [18]:
def concat_previous(str_list):
    result = ""
    for s in str_list:
        clean_str = s.strip()
        if result and not result.endswith((' ', '.', ',')):
            result += " "
        result += clean_str
    return result


In [19]:
def process_record(d, mode="dummy", numSents=3):
    previous = None
    for file in sorted(os.listdir(d), key=extract_number):
        pattern = r'audio-(\d+)\.wav'
        match = re.match(pattern, file)
        if match:
            data = os.path.join(d, file)
            prompt = concat_previous(previous) if previous is not None and mode != 'dummy' else None
            print('> File:', file, '\prompt:', prompt, '\n[', previous, ']')
            segments, info = model.transcribe(data, beam_size=5, initial_prompt=prompt)
            target_file = data.replace('.wav', f'-{mode}.txt')
            with open(target_file, 'w') as file:
                pass  
            with open(target_file, 'a') as in_file:
                concat = ""
                for segment in segments:
                    concat += segment.text

                if (not isinstance(previous, list)):
                        previous = [concat]
                else:
                    if (len(previous) == numSents):
                        previous = [concat] + previous[:-1]
                    else:
                        previous.append(concat)
                print(concat, file=in_file)


In [20]:
def process_all():
    input_prefix = "/Users/karelvlk/Developer/mff/ufal/whisper-prompting/ESICv1.0/dev/"
    for folder in os.listdir(input_prefix):
        if os.path.isdir(input_prefix + folder):
            for subfolder in os.listdir(input_prefix + folder):
                if os.path.isdir(input_prefix + folder + "/" + subfolder):
                    for file in os.listdir(input_prefix + folder + "/" + subfolder):
                        if file == "en.IStde.man.vert+ts":
                            input_file_d = input_prefix + folder + "/" + subfolder
                            process_record(input_file_d)


In [21]:

process_record('/Users/karelvlk/Developer/mff/ufal/whisper-prompting/ESICv1.0/dev/20080901/018_006_EN_Ždanoka', 'one', 1)
process_record('/Users/karelvlk/Developer/mff/ufal/whisper-prompting/ESICv1.0/dev/20080901/018_006_EN_Ždanoka', 'two', 2)
process_record('/Users/karelvlk/Developer/mff/ufal/whisper-prompting/ESICv1.0/dev/20080901/018_006_EN_Ždanoka', 'three', 3)
process_record('/Users/karelvlk/Developer/mff/ufal/whisper-prompting/ESICv1.0/dev/20080901/018_006_EN_Ždanoka', 'four', 4)
process_record('/Users/karelvlk/Developer/mff/ufal/whisper-prompting/ESICv1.0/dev/20080901/018_006_EN_Ždanoka', 'five', 5)
process_record('/Users/karelvlk/Developer/mff/ufal/whisper-prompting/ESICv1.0/dev/20080901/018_006_EN_Ždanoka', 'size', 6)
process_record('/Users/karelvlk/Developer/mff/ufal/whisper-prompting/ESICv1.0/dev/20080901/018_006_EN_Ždanoka', 'seven', 7)
process_record('/Users/karelvlk/Developer/mff/ufal/whisper-prompting/ESICv1.0/dev/20080901/018_006_EN_Ždanoka', 'all', 100000000000000000000)


> File: audio-1.wav \prompt: None 
[ None ]
> File: audio-2.wav \prompt: On behalf of the Green Zephyr Group, I would like to thank Mr. Brake for his great report. 
[ [' On behalf of the Green Zephyr Group, I would like to thank Mr. Brake for his great report.'] ]
> File: audio-3.wav \prompt: And we appreciate his point that consulting the VISA using the number of the VISA sticker in combination with the verification of fingerprints will create a lot of problems. 
[ [' And we appreciate his point that consulting the VISA using the number of the VISA sticker in combination with the verification of fingerprints will create a lot of problems.'] ]
> File: audio-4.wav \prompt: And therefore, we welcome introducing of derogation in exceptional cases, consulting the VISA without verification of fingerprints. 
[ [' And therefore, we welcome introducing of derogation in exceptional cases, consulting the VISA without verification of fingerprints.'] ]
> File: audio-5.wav \prompt: Nevertheless, in